# Task 2: Implement decison tree

Dataset: LendingClub Safe Loan  
Implement information gain  
Implement information gain ratio (optional)  

Using 10-fold cross validation for evaluation  
Evaluation matrices: Accuracy, Precision, Recall, F1  
Choosing deferent maximum depth and check the performance  
Plotting figure to demonstrate deferent accuracy in terms of different maximum depth (From 1 to 10)   


## 1. Import dataset

In [ ]:
# import library
import pandas as pd
import numpy as np
import json

In [ ]:
# read dataset
loans = pd.read_csv('data/lendingclub/lending-club-data.csv', low_memory=False)

In [ ]:
# Using safe_loan for target, and setting positive sample to 1, negative sample to -1, reomve bad_loan
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
del loans['bad_loans']

In [ ]:
#using four features "grade, term, home_ownership, emp_length"
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

In [ ]:
#check the dataset
loans.head()

## 2. Split dataset

In [ ]:
from sklearn.utils import shuffle
loans = shuffle(loans, random_state = 34)

split_line = int(len(loans) * 0.6)
train_data = loans.iloc[: split_line]
test_data = loans.iloc[split_line:]

## 3. Feature preprocessing

All of the features are descrate, using one-hot encoding

In [ ]:
#Using pd.get_dummies to obtain one-hot encoding
def one_hot_encoding(data, features_categorical):
    '''
    Parameter
    ----------
    data: pd.DataFrame
    
    features_categorical: list(str)
    '''
    
    for cat in features_categorical:
        
        one_encoding = pd.get_dummies(data[cat], prefix = cat)
        
        data = pd.concat([data, one_encoding],axis=1)
        
        del data[cat]
    
    return data

In [ ]:
train_data = one_hot_encoding(train_data, features)

In [ ]:
train_data.head()

In [ ]:
#obtain feature name
one_hot_features = train_data.columns.tolist()
one_hot_features.remove(target)
one_hot_features

In [ ]:
#obtain one hot encoding for test data
test_data_tmp = one_hot_encoding(test_data, features)

In [ ]:
test_data = pd.DataFrame(columns = train_data.columns)
for feature in train_data.columns:
    if feature in test_data_tmp.columns:
        test_data[feature] = test_data_tmp[feature].copy()
    else:
        test_data[feature] = np.zeros(test_data_tmp.shape[0], dtype = 'uint8')

In [ ]:
test_data.head()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

Training set contains 37224 samples and testing set has 9284 samples  
All feature is one-hot encoding  
targer lable is 1 or -1

In [ ]:
## 4. Implement Information Gain

Information Entropy：
$$
\mathrm{Ent}(D) = - \sum^{\vert \mathcal{Y} \vert}_{k = 1} p_k \mathrm{log}_2 p_k
$$

Information Gain：
$$
\mathrm{Gain}(D, a) = \mathrm{Ent}(D) - \sum^{V}_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Ent}(D^v)
$$

if $p = 0$，$p \log_2p = 0$

In [ ]:
def information_entropy(labels_in_node):
    '''
    calculating information entropy for node
    
    Parameter
    ----------
    labels_in_node: np.ndarray, for example [-1, 1, -1, 1, 1]
    
    Returns
    ----------
    float: information entropy
    '''
    # number of samples
    num_of_samples = labels_in_node.shape[0]
    
    if num_of_samples == 0:
        return 0
    
    # number of positive samples
    num_of_positive = len(labels_in_node[labels_in_node == 1])
    
    # number of negative samples
    num_of_negative =                                                                     # YOUR CODE HERE
    
    # the probability of positive sample
    prob_positive = num_of_positive / num_of_samples
    
    # the probability of negative sample
    prob_negative =                                                                       # YOUR CODE HERE
    
    if prob_positive == 0:
        positive_part = 0
    else:
        positive_part = prob_positive * np.log2(prob_positive)
    
    if prob_negative == 0:
        negative_part = 0
    else:
        negative_part = prob_negative * np.log2(prob_negative)
    
    return - ( positive_part + negative_part )

In [ ]:
# test case 1
example_labels = np.array([-1, -1, 1, 1, 1])
print(information_entropy(example_labels)) # 0.97095

# test case2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
print(information_entropy(example_labels)) # 0.86312
    
# test case3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
print(information_entropy(example_labels)) # 0.86312

# test case4
example_labels = np.array([-1] * 9 + [1] * 8)
print(information_entropy(example_labels)) # 0.99750

# test case5
example_labels = np.array([1] * 8)
print(information_entropy(example_labels)) # 0

# test case6
example_labels = np.array([])
print(information_entropy(example_labels)) # 0

In [ ]:
#compute information gains
def compute_information_gains(data, features, target, annotate = False):
    ''' 
    Parameter
    ----------
        data: pd.DataFrame，sample
        
        features: list(str)，feature name
        
        target: str, lable name
        
        annotate, boolean，print information gains for all features? False by default
        
    Returns
    ----------
        information_gains: dict, key: str, feature name
                                 value: float，information gains
    '''
    

    information_gains = dict()
    
    # Fore each feature, compute information gains
    for feature in features:
        

        left_split_target = data[data[feature] == 0][target]
        
        right_split_target =  data[data[feature] == 1][target]
            
        # compute inforamtion entropy for left subtree
        left_entropy = information_entropy(left_split_target)
        
        # compute weight for left subtree
        left_weight = len(left_split_target) / (len(left_split_target) + len(right_split_target))

       # compute inforamtion entropy for right subtree
        right_entropy =                                                                 # YOUR CODE HERE
        
        # compute weight for right subtree
        right_weight =                                                                  # YOUR CODE HERE
        
        # compute informatino entropy for data
        current_entropy = information_entropy(data[target])
            
        # compute information gains based on feature
        gain =                                                                             # YOUR CODE HERE
        
        # store information gains
        information_gains[feature] = gain
        
        if annotate:
            print(" ", feature, gain)
            
    return information_gains

In [ ]:
# test case 1
print(compute_information_gains(train_data, one_hot_features, target)['grade_A']) # 0.01759

# test case 2
print(compute_information_gains(train_data, one_hot_features, target)['term_ 60 months']) # 0.01429

# test case 3
print(compute_information_gains(train_data, one_hot_features, target)['grade_B']) # 0.00370

### 4.2 Information Gain Ratio
Information gain ratio：

$$
\mathrm{Gain\_ratio}(D, a) = \frac{\mathrm{Gain}(D, a)}{\mathrm{IV}(a)}
$$

where

$$
\mathrm{IV}(a) = - \sum^V_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \log_2 \frac{\vert D^v \vert}{\vert D \vert}
$$

### 4.3 Gini
Gini：

$$
\begin{aligned}
\mathrm{Gini}(D) & = \sum^{\vert \mathcal{Y} \vert}_{k=1} \sum_{k' \neq k} p_k p_{k'}\\
& = 1 - \sum^{\vert \mathcal{Y} \vert}_{k=1} p^2_k.
\end{aligned}
$$

Gini index for feature $a$：

$$
\mathrm{Gini\_index}(D, a) = \sum^V_{v = 1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Gini}(D^v)
$$

## 5. Chose the best feature 

In [ ]:
def best_splitting_feature(data, features, target, criterion = 'information_gain', annotate = False):
    '''    
    Parameters
    ----------
    data: pd.DataFrame, 
    
    features: list(str)，
    
    target: str， 
    
    criterion: str, 
    
    annotate: boolean, default False，
    
    Returns
    ----------
    best_feature: str, 
    
    '''
    if criterion == 'information_gain':
        if annotate:
            print('using information gain')
        
        # obtain the information gains
        information_gains = compute_information_gains(data, features, target, annotate)
    
        # Select the best feature according to the maximum information gains
        best_feature =                                                                      # YOUR CODE HERE
        
        return best_feature

    else:
        raise Exception("criterion is abnormal!", criterion)

## 6. Obtain the number of sample for minority class

In [ ]:
#Obtain the number of sample for minority class
def intermediate_node_num_mistakes(labels_in_node):
    '''
    return the number of sample for minority class，For example [1, 1, -1, -1, 1]，return 2
    
    Parameter
    ----------
    labels_in_node: np.ndarray, pd.Series
    
    Returns
    ----------
    int：
    
    '''
    if len(labels_in_node) == 0:
        return 0
    
    num_of_one =                                                                          # YOUR CODE HERE
    
    num_of_minus_one =                                                                # YOUR CODE HERE
    
    return num_of_one if num_of_minus_one > num_of_one else num_of_minus_one

In [ ]:
# test case 1
print(intermediate_node_num_mistakes(np.array([1, 1, -1, -1, -1]))) # 2

# test case 2
print(intermediate_node_num_mistakes(np.array([]))) # 0

# test3
print(intermediate_node_num_mistakes(np.array([1]))) # 0

## 7. Create leaf node

In [ ]:
def create_leaf(target_values):
    '''
    Compute the target value of corresponding leaf node, and store node information into dict
   
    
    Parameter:
    ----------
    target_values: pd.Series, 

    Returns:
    ----------
    leaf: dict，leaf node，
            leaf['splitting_features'], None，No need to split feature
            leaf['left'], None，leaf node do not have left subtree 
            leaf['right'], None，leaf node do not have right subtree
            leaf['is_leaf'], True, is leaf node?
            leaf['prediction'], int, the predicton of leaf node
    '''
    # create leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}
   
    # obtain the numbe of positive samples and negative samples
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    if num_ones > num_minus_ones:
        leaf['prediction'] = 1
    else:
        leaf['prediction'] = -1

    # 返回叶子结点
    return leaf

## 8. Building decision tree
 
Stop conditions：
1. all of the samples have the same lable 
2. all features have been used 
3. the depth of node is maximum depth

In [ ]:
def decision_tree_create(data, features, target, criterion = 'information_gain', current_depth = 0, max_depth = 10, annotate = False):
    '''
    Parameter:
    ----------
    data: pd.DataFrame, 

    features: iterable, 

    target: str, 

    criterion: 'str', 'information_gain'

    current_depth: int, 

    max_depth: int, 

    Returns:
    ----------
    dict, dict['is_leaf']          : False,
          dict['prediction']       : None,
          dict['splitting_feature']: splitting_feature, 
          dict['left']             : dict
          dict['right']            : dict
    '''
    
    if criterion not in ['information_gain', 'gain_ratio', 'gini']:
        raise Exception("criterion is abnormal!", criterion)
    
    remaining_features = features[:]
    
    target_values = data[target]
    print("-" * 50)
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))

    # Stop condition 1
    # all of the samples have the same lable
    # Using function intermediate_node_num_mistakes
    if                                                                                  # YOUR CODE HERE
        print("Stopping condition 1 reached.")
        return create_leaf(target_values) 
    
    
    # Stop condition 2
    # all features have been used，remaining_features is null 
    if                                                                                  # YOUR CODE HERE
        print("Stopping condition 2 reached.")
        return create_leaf(target_values)  
    
    # Stop condition 3
    # the depth of node is maximum depth
    
    if                                                                                  # YOUR CODE HERE
        print("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values)   

    # obtain the optimal feature for splitting
    # Using function best_splitting_feature 
    
    splitting_feature =                                                                 # YOUR CODE HERE
    
    # split data
    # left subtree
    left_split = data[data[splitting_feature] == 0]
    
    # right subtree
    right_split =                                                                       # YOUR CODE HERE
    
    # remove optimal feature from ramaining features
    remaining_features.remove(splitting_feature)
    
    # print splitting feature and the number of samples of left tree and right tree 
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # If all of the samples are splited into one subtree, create left node 
    # left tree
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    
    # right tree
    if len(right_split) == len(data):
        print("Creating right node.")
        return                                                                          # YOUR CODE HERE

    # create left tree recursively 
    left_tree = decision_tree_create(left_split, remaining_features, target, criterion, current_depth + 1, max_depth, annotate)
    
    # create right tree recursively 
    
    right_tree =                                                                        # YOUR CODE HERE

    # return nodes except left nodes
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [ ]:
#create a decision tree
my_decision_tree = decision_tree_create(train_data, one_hot_features, target, 'gini', max_depth = 6, annotate = False)

## 9. Prediction

In [ ]:
def classify(tree, x, annotate = False):
    '''
    
    Parameters
    ----------
    tree: dict
    
    x: pd.Series，sample to be predicted
    
    annotate： boolean, 
    
    Returns
    ----------
    return the predicted lable
    '''
    if tree['is_leaf']:
        if annotate:
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [ ]:
test_sample = test_data.iloc[0]
print(test_sample)

In [ ]:
print('True class: %s ' % (test_sample['safe_loans']))
print('Predicted class: %s ' % classify(my_decision_tree, test_sample))

In [ ]:
#print the process of decision tree
classify(my_decision_tree, test_sample, annotate=True)

## 10. Evaluate the model in testing set

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
def predict(tree, data):
    '''
    
    Parameter
    ----------
    tree: dict, decision tree model
    
    data: pd.DataFrame, data
    
    Returns
    ----------
    predictions：np.ndarray, the predicted lables
    '''
    predictions = np.zeros(len(data)) 
    
    predictions = classify(tree,)
    # YOUR CODE HERE
    
    return predictions

## 11. Evaluate decision tree using information gain


In [ ]:
# Note that the maximum depth is 6
# YOUR CODE HERE


the maximum depth is 6

###### Input your results

|Accuracy|Precision|Recall|F1
-|-|-|-|-
Information Gain|0.0|0.0|0.0|0.0
